<a href="https://colab.research.google.com/github/prateek21432/fracture-pred/blob/main/fracturedet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
from PIL import Image, ImageFile

# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Path to dataset directories
train_dir = "/content/drive/MyDrive/Bone_Fracture_Binary_Classification/train"
val_dir = "/content/drive/MyDrive/Bone_Fracture_Binary_Classification/val"
test_dir = "/content/drive/MyDrive/Bone_Fracture_Binary_Classification/test"

# Function to check and remove corrupted images
def remove_corrupted_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Check if the image is valid
            except (IOError, SyntaxError) as e:
                print(f"Removing corrupted image: {file_path}")
                os.remove(file_path)

# Remove corrupted images from all datasets
remove_corrupted_images(train_dir)
remove_corrupted_images(val_dir)
remove_corrupted_images(test_dir)

# ImageDataGenerator for loading and augmenting images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images using generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile Model
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train Model
epochs = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

# Evaluate Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save Model
model.save("/content/drive/MyDrive/Bone_Fracture_Binary_Classification/model/model.h5")
print("Model saved successfully.")



Found 9246 images belonging to 2 classes.
Found 839 images belonging to 2 classes.
Found 506 images belonging to 2 classes.
Epoch 1/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 157s 531ms/step - accuracy: 0.5633 - loss: 0.6808 - val_accuracy: 0.6830 - val_loss: 0.6112
Epoch 2/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 150s 519ms/step - accuracy: 0.6506 - loss: 0.6207 - val_accuracy: 0.7294 - val_loss: 0.5623
Epoch 3/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 164s 569ms/step - accuracy: 0.7010 - loss: 0.5768 - val_accuracy: 0.7175 - val_loss: 0.5713
Epoch 4/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 189s 524ms/step - accuracy: 0.7125 - loss: 0.5540 - val_accuracy: 0.7747 - val_loss: 0.4785
Epoch 5/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 151s 522ms/step - accuracy: 0.7436 - loss: 0.5227 - val_accuracy: 0.6830 - val_loss: 0.6208
Epoch 6/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 150s 520ms/step - accuracy: 0.7570 - loss: 0.5047 - val_accuracy: 0.7914 - val_loss: 0.4647
Epoch 7/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 153s 530ms/step - accuracy: 0.7584 - loss: 0

Test Accuracy: 0.7708
Model saved successfully.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
from PIL import Image
from PIL import ImageFile

# Allow PIL to load truncated images instead of throwing an error
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load the trained model
model = load_model("/content/drive/MyDrive/Bone_Fracture_Binary_Classification/model/model.h5")

# Define the path to the test images
test_dir = "/content/drive/MyDrive/Bone_Fracture_Binary_Classification/testing"

# Define image size (same as used during training)
image_size = (224, 224)

# Get class labels (assuming binary classification: 'Fractured' and 'Normal')
class_labels = ["Fractured", "Normal"]

# Function to preprocess and predict an image
def predict_image(image_path):
    try:
        # Load image
        img = Image.open(image_path)
        img = img.convert("RGB")  # Ensure it's in RGB format
        img = img.resize(image_size)  # Resize
        img_array = img_to_array(img) / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img_array)[0][0]

        # Convert prediction to class label
        class_index = 1 if prediction >= 0.5 else 0
        confidence = prediction if prediction >= 0.5 else 1 - prediction
        return class_labels[class_index], confidence
    except Exception as e:
        print(f"Skipping corrupted image: {image_path} - Error: {e}")
        return None, None

# Iterate over test images and classify them
print("Testing Images...\n")
for root, _, files in os.walk(test_dir):
    for file in files:
        image_path = os.path.join(root, file)
        label, confidence = predict_image(image_path)
        if label is not None:  # Skip corrupted images
            print(f"Image: {file} → Predicted: {label} (Confidence: {confidence:.2f})")


Testing Images...

